In [1]:
#!pip3 install pymongo
import csv 
import json, requests
import pandas as pd
import time
import numpy as np
import psycopg2, os
from flask import Flask, request, render_template

In [2]:
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27")
db = client.apan5400
col = db.movie_reviews
print(col.count_documents({}))

932


In [3]:
#for recommendation
import difflib
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Get movies from database

In [4]:
#USING DATABASE
tmdb = db.tmdb
#print(tmdb.count_documents({}))
df2 = pd.DataFrame(list(tmdb.find()))

#USING CSV
#df2 = pd.read_csv('tmdb_dt.csv')
movie_title = df2.title
df2 = df2.drop_duplicates(subset = ["title"])

In [5]:
print(df2)

                          _id                                     genres  \
0    610b1ca2711735c4ba85c71c         ['action', 'adventure', 'fantasy']   
1    610b1ca2711735c4ba85c71d           ['action', 'adventure', 'crime']   
2    610b1ca2711735c4ba85c71e  ['action', 'adventure', 'sciencefiction']   
3    610b1ca2711735c4ba85c71f                    ['animation', 'family']   
4    610b1ca2711735c4ba85c720         ['adventure', 'fantasy', 'family']   
..                        ...                                        ...   
244  610b1ca2711735c4ba85c810        ['action', 'sciencefiction', 'war']   
245  610b1ca2711735c4ba85c811              ['drama', 'music', 'romance']   
246  610b1ca2711735c4ba85c812                     ['drama', 'adventure']   
247  610b1ca2711735c4ba85c813         ['fantasy', 'action', 'adventure']   
248  610b1ca2711735c4ba85c814      ['adventure', 'fantasy', 'animation']   

         id                                           keywords  \
0     19995          

In [6]:
count = CountVectorizer(stop_words='english')
#count_matrix = count.fit_transform(df2['soup'])
count_matrix = count.fit_transform(df2['soup'].values.astype('U')) 

#x = v.fit_transform(df2['soup'].values.astype('U')) 

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])
all_titles = [df2['title'][i] for i in range(len(df2['title']))]

print(all_titles)

['The Dark Knight', 'Inception', 'Parasite', 'Se7en', 'The Silence of the Lambs', 'The Prestige', 'The Departed', 'The Usual Suspects', 'L√©on: The Professional', 'Psycho', 'Rear Window', 'Joker', 'The Lives of Others', 'Oldboy', 'Memento', 'Witness for the Prosecution', '1917', 'Reservoir Dogs', 'Come and See', 'Aliens', 'Das Boot', 'North by Northwest', 'Vertigo', 'Double Indemnity', 'M', 'Andhadhun', 'Drishyam', 'Gangs of Wasseypur', 'The Secret in Their Eyes', 'Shutter Island', 'L.A. Confidential', 'The Bandit', 'Heat', 'Die Hard', 'Sholay', 'Chinatown', 'The Great Escape', 'Yojimbo', 'The Seventh Seal', 'Dial M for Murder', 'The Invisible Guest', 'The Handmaiden', 'Haider', 'Logan', 'Room', 'Wild Tales', 'Gone Girl', 'Kahaani', 'Prisoners', 'Mad Max: Fury Road', 'A Wednesday', 'No Country for Old Men', 'V for Vendetta', 'Memories of Murder', 'Kill Bill: Vol. 1', 'Amores Perros', 'The Sixth Sense', 'Fargo', 'Jurassic Park', 'Blade Runner', 'Persona', 'What Ever Happened to Baby Jan

In [7]:
def get_recommendations(title):
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    tit = df2['title'].iloc[movie_indices]
    #dat = df2['release_date'].iloc[movie_indices]
    #return_df = pd.DataFrame(columns=['Title','Year'])
    return_df = pd.DataFrame(columns=['Title'])
    return_df['Title'] = tit
    #return_df['Year'] = dat
    return return_df

In [8]:
    #for recommendation
    m_name = 'The Dark Knight'
    result_final = get_recommendations(m_name)
    print (result_final)
    names = []
    for i in range(len(result_final)):
        names.append(result_final.iloc[i][0])
        

                  Title
104              Master
237    Midnight Express
5          The Prestige
6          The Departed
7    The Usual Suspects
10          Rear Window
20             Das Boot
24                    M
26             Drishyam
48            Prisoners


In [9]:
print (names)

['Master', 'Midnight Express', 'The Prestige', 'The Departed', 'The Usual Suspects', 'Rear Window', 'Das Boot', 'M', 'Drishyam', 'Prisoners']


## Using Flask to render the HTML

In [ ]:
app = Flask("MyApp")

@app.route('/')
def my_form():
    return render_template("main.html")

@app.route('/', methods=['POST'])
def my_form_post():
    val = request.form['userinput']
    query = {"movie": {"$regex": val, "$options" : "i"} }
    docs = col.find(query).sort("movie", 1)
    title = []
    review = []
    sentiment = []
    source = []
    recom = []
    negative_sentiment = []
  
    #for recommendation test
    #m_name = 'The Dark Knight'
    #print(val)
    result_final = get_recommendations(val)
    #print (result_final)
    names = []
    dates = []
    for i in range(len(result_final)):
        names.append(result_final.iloc[i][0])

    for doc in docs:
        title.append(doc['movie']) 
        review.append(doc['user_review'])
        sentiment.append(doc['sentiment'])
        source.append(doc['source'])
        negative_sentiment.append(doc['negative sentiment'])
        recom.append(names)
        
        #for recommendation
        #result_final = get_recommendations(title)
        #print (result_final)
        #names_rec = []
        #for i in range(len(result_final)):
        #    names_rec.append(result_final.iloc[i][0])
        #recom.append(names_rec)
        
    df = pd.DataFrame({
        'Movie Title': title,
        'Movie Review': review,
        'Sentiment': sentiment
        'Negative Sentiment': negative_sentiment,
        'Movie Recommendation': recom,
        'Source': source,
    })
        
    return render_template("results.html", keyword = val, results =list(df.values.tolist()))

app.run(host='localhost', port=5001)

 * Serving Flask app 'MyApp' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Aug/2021 19:42:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2021 19:42:58] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2021 19:43:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2021 19:43:11] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2021 19:44:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2021 19:44:26] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2021 19:44:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2021 19:44:39] "POST / HTTP/1.1" 200 -
